<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

<br>

# <font color="#76b900">**Notebook 7:** Retrieval-Augmented Generation with Vector Stores</font>

<br>

In the previous notebook, we learned about embedding models and exercised some of their capabilities. We discussed their intended use cases of longer-form document comparison and found ways to use it as a backbone for more custom semantic comparisons. This notebook will progress these ideas toward the retrieval model's intended use case and explore how to build chatbot systems that rely on *vector stores* to automatically save and retrieve information.

<br>

### **Learning Objectives:**

- Understand how semantic-similarity-backed systems can facilitate easy-to-use retrieval formulations.

- Learn how to incorporate retrieval modules into your chat model systems for a retrieval-augmented generation (RAG) pipeline, which can be applied to tasks like document retrieval and conversation memory buffers.

<br>

### **Questions To Think About:**

- This notebook does not attempt to incorporate hierarchical reasoning or non-naive RAG (such as planning agents). Consider what modifications would be necessary to make these components work in an LCEL chain.

- Consider when it would be best to move your vector store solution into a scalable service and when a GPU will become necessary for optimization.

<br>

### **Environment Setup:**

In [1]:
# %%capture
## ^^ Comment out if you want to see the pip install process

## Necessary for Colab, not necessary for course environment
# %pip install -q langchain langchain-nvidia-ai-endpoints gradio rich
# %pip install -q arxiv pymupdf faiss-cpu

## If you encounter a typing-extensions issue, restart your runtime and try again
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# ChatNVIDIA.get_available_models()

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

In [2]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="deepseek-ai/deepseek-r1-distill-qwen-32b")

In [3]:
pprint(ChatNVIDIA.get_available_models())

/usr/local/lib/python3.11/site-packages/langchain_nvidia_ai_endpoints/_common.py:237: UserWarning: Default model is set as: 01-ai/yi-large. 
Set model using model parameter. 
To get available models use available_models property.
  warnings.warn(


[
    Model(
        id='01-ai/yi-large',
        model_type='chat',
        client='ChatNVIDIA',
        endpoint=None,
        aliases=['ai-yi-large'],
        supports_tools=False,
        supports_structured_output=False,
        base_model=None
    ),
    Model(
        id='abacusai/dracarys-llama-3.1-70b-instruct',
        model_type='chat',
        client='ChatNVIDIA',
        endpoint=None,
        aliases=None,
        supports_tools=False,
        supports_structured_output=False,
        base_model=None
    ),
    Model(
        id='ai21labs/jamba-1.5-large-instruct',
        model_type='chat',
        client='ChatNVIDIA',
        endpoint=None,
        aliases=None,
        supports_tools=False,
        supports_structured_output=False,
        base_model=None
    ),
    Model(
        id='ai21labs/jamba-1.5-mini-instruct',
        model_type='chat',
        client='ChatNVIDIA',
        endpoint=None,
        aliases=None,
        supports_tools=False,
        supports_structured_output=False,
        base_model=None
    ),
    Model(
        id='aisingapore/sea-lion-7b-instruct',
        model_type='chat',
        client='ChatNVIDIA',
        endpoint=None,
        aliases=['ai-sea-lion-7b-instruct'],
        supports_tools=False,
        supports_structured_output=False,
        base_model=None
    ),
    Model(
        id='baichuan-inc/baichuan2-13b-chat',
        model_type='chat',
        client='ChatNVIDIA',
        endpoint=None,
        aliases=None,
        supports_tools=False,
        supports_structured_output=False,
        base_model=None
    ),
    Model(
        id='databricks/dbrx-instruct',
        model_type='chat',
        client='ChatNVIDIA',
        endpoint=None,
        aliases=['ai-dbrx-instruct'],
        supports_tools=False,
        supports_structured_output=False,
        base_model=None
    ),
    Model(
        id='deepseek-ai/deepseek-coder-6.7b-instruct',
        model_type='chat',
        client='ChatNVIDIA',
        endpoint=None,
        aliases=['ai-deepseek-coder-6_7b-instruct'],
        supports_tools=False,
        supports_structured_output=False,
        base_model=None
    ),
    Model(
        id='deepseek-ai/deepseek-r1',
        model_type='chat',
        client='ChatNVIDIA',
        endpoint=None,
        aliases=None,
        supports_tools=False,
        supports_structured_output=False,
        base_model=None
    ),
    Model(
        id='deepseek-ai/deepseek-r1-distill-llama-8b',
        model_type=None,
        client=None,
        endpoint=None,
        aliases=None,
        supports_tools=False,
        supports_structured_output=False,
        base_model=None
    ),
    Model(
        id='deepseek-ai/deepseek-r1-distill-qwen-14b',
        model_type=None,
        client=None,
        endpoint=None,
        aliases=None,
        supports_tools=False,
        supports_structured_output=False,
        base_model=None
    ),
    Model(
        id='deepseek-ai/deepseek-r1-distill-qwen-32b',
        model_type=None,
        client=None,
        endpoint=None,
        aliases=None,
        supports_tools=False,
        supports_structured_output=False,
        base_model=None
    ),
    Model(
        id='deepseek-ai/deepseek-r1-distill-qwen-7b',
        model_type=None,
        client=None,
        endpoint=None,
        aliases=None,
        supports_tools=False,
        supports_structured_output=False,
        base_model=None
    ),
    Model(
        id='google/codegemma-1.1-7b',
        model_type='chat',
        client='ChatNVIDIA',
        endpoint=None,
        aliases=['ai-codegemma-1.1-7b'],
        supports_tools=False,
        supports_structured_output=False,
        base_model=None
    ),
    Model(
        id='google/codegemma-7b',
        model_type='chat',
[1;3

----

<br>

## Part 1: Summary of RAG Workflows

This notebook will explore several paradigms and derive reference code to help you approach some of the most common retrieval-augmented workflows. Specifically, the following sections will be covered (with the differences highlighted):

<br>

> ***Vector Store Workflow for Conversational Exchanges:***
- Generate semantic embedding for each new conversation.
- Add the message body to a vector store for retrieval.
- Query the vector store for relevant messages to fill in the LLM context.

<br>

> ***Modified Workflow for an Arbitrary Document:***
- **Divide the document into chunks and process them into useful messages.**
- Generate semantic embedding for each **new document chunk**.
- Add the **chunk bodies** to a vector store for retrieval.
- Query the vector store for relevant **chunks** to fill in the LLM context.
    - ***Optional:* Modify/synthesize results for better LLM results.**

<br>

> **Extended Workflow for a Directory of Arbitrary Documents:**
- Divide **each document** into chunks and process them into useful messages.
- Generate semantic embedding for each new document chunk.
- Add the chunk bodies to **a scalable vector database for fast retrieval**.
    - ***Optional*: Exploit hierarchical or metadata structures for larger systems.**
- Query the **vector database** for relevant chunks to fill in the LLM context.
    - *Optional:* Modify/synthesize results for better LLM results.

<br>

Some of the most important terminology surrounding RAG is covered in detail on the [**LlamaIndex Concepts page**](https://docs.llamaindex.ai/en/stable/getting_started/concepts.html), which itself is a great starting point for progressing towards the LlamaIndex loading and retrieving strategy. We highly recommend using it as a reference as you continue with this notebook and advise you to try out LlamaIndex after the course to consider the pros and cons firsthand!


<!-- > <img src="https://drive.google.com/uc?export=view&id=1cFbKbVvLLnFPs3yWCKIuzXkhBWh6nLQY" width=1200px/> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/data_connection_langchain.jpeg" width=1200px/>
>
> From [**Retrieval | LangChain**🦜️🔗](https://python.langchain.com/v0.1/docs/modules/data_connection/)

----

<br>

## **Part 2:** RAG for Conversation History

In our previous explorations, we delved into the capabilities of document embedding models and used them to embed, store, and compare semantic vector representations of text. Though we could motivate how to efficiently extend this into vector store land manually, the true beauty of working with a standard API is its strong incorporation with other frameworks that can already do the heavy lifting for us!

<br>

### **Step 1**: Getting A Conversation

Consider a conversation crafted using Llama-13B between a chat agent and a blue bear named Beras. This dialogue, dense with details and potential diversions, provides a rich dataset for our study:


In [4]:
conversation = [  ## This conversation was generated partially by an AI system, and modified to exhibit desirable properties
    "[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the rocky mountains?",
    "[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch across North America",
    "[Beras] Wow, that sounds amazing! Ive never been to the Rocky Mountains before, but Ive heard many great things about them.",
    "[Agent] I hope you get to visit them someday, Beras! It would be a great adventure for you!",
    "[Beras] Thank you for the suggestion! Ill definitely keep it in mind for the future.",
    "[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research online or watching documentaries about them.",
    "[Beras] I live in the arctic, so I'm not used to the warm climate there. I was just curious, ya know!",
    "[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains and their significance!"
]

Using the manual embedding strategy from the previous notebook is still very viable, but we can also rest easy and let a **vector store** do all that work for us!

<br>

### **Step 2:** Constructing Our Vector Store Retriever

To streamline similarity queries on our conversation, we can employ a vector store to help keep track of passages for us! **Vector Stores**, or vector storage systems, abstract away most of the low-level details of the embedding/comparison strategies and provide a simple interface to load and compare vectors.


<!-- > <img src="https://drive.google.com/uc?export=view&id=1ZjwYbSZzsXK6ZP8O1-cY3BeRffV4oqzb" width=1000px/> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/vector_stores.jpeg" width=1200px/>
>
> From [**Vector Stores | LangChain**🦜️🔗](https://python.langchain.com/v0.1/docs/modules/data_connection/vectorstores/)

<br>

In addition to simplifying the process from an API perspective, vector stores also implement connectors, integrations, and optimizations under the hood. In our case, we will start with the [**FAISS vector store**](https://python.langchain.com/docs/integrations/vectorstores/faiss), which integrates a LangChain-compatable Embedding model with the [**FAISS (Facebook AI Similarity Search)**](https://github.com/facebookresearch/faiss) library to make the process fast and scalable on our local machine!

**Specifically:**

1. We can feed our conversation into [**a FAISS vector store**](https://python.langchain.com/docs/integrations/vectorstores/faiss) via the `from_texts` constructor. This will take our conversational data and the embedding model to create a searchable index over our discussion.
2. This vector store can then be "interpreted" as a retriever, supporting the LangChain runnable API and returning documents retrieved via an input query.

The following shows how you can construct a FAISS vector store and reinterpret it as a retriever using the LangChain `vectorstore` API:

In [5]:
%%time
## ^^ This cell will be timed to see how long the conversation embedding takes
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain.vectorstores import FAISS

## Streamlined from_texts FAISS vectorstore construction from text list
convstore = FAISS.from_texts(conversation, embedding=embedder)
retriever = convstore.as_retriever()

CPU times: user 175 ms, sys: 376 ms, total: 551 ms
Wall time: 744 ms


The retriever can now be used like any other LangChain runnable to query the vector store for some relevant documents:

In [6]:
pprint(retriever.invoke("What is your name?"))

[
    Document(
        id='065abfa9-c315-4432-875a-c84c0762453b',
        metadata={},
        page_content="[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the 
rocky mountains?"
    ),
    Document(
        id='34298977-1d00-4080-8fe8-ced9a8bed0e8',
        metadata={},
        page_content='[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains
and their significance!'
    ),
    Document(
        id='c634c15e-9faa-4e43-9700-b99b990f1d1e',
        metadata={},
        page_content='[Agent] I hope you get to visit them someday, Beras! It would be a great adventure for you!'
    ),
    Document(
        id='79217739-75be-418b-800a-3f40170633c5',
        metadata={},
        page_content="[Beras] I live in the arctic, so I'm not used to the warm climate there. I was just curious, 
ya know!"
    )
]

In [7]:
pprint(retriever.invoke("Where are the Rocky Mountains?"))

[
    Document(
        id='7524d013-5a56-458f-ac66-5ea4dc14bb6b',
        metadata={},
        page_content='[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch 
across North America'
    ),
    Document(
        id='065abfa9-c315-4432-875a-c84c0762453b',
        metadata={},
        page_content="[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the 
rocky mountains?"
    ),
    Document(
        id='8f89a3fe-e466-4d49-8ea3-e2288c938f1f',
        metadata={},
        page_content='[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research 
online or watching documentaries about them.'
    ),
    Document(
        id='34298977-1d00-4080-8fe8-ced9a8bed0e8',
        metadata={},
        page_content='[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains
and their significance!'
    )
]

As we can see, our retriever found a handful of semantically relevant documents from our query. You may notice that not all of the documents are useful or clear on their own. For example, a retrieval of *"Beras"* for *"your name"* may be problematic for the chatbot if provided out of context. Anticipating the potential problems and creating synergies between your LLM components can increase the likelihood of good RAG behavior, so keep an eye out for such pitfalls and opportunities.

<br>

### **Step 3:** Incorporating Conversation Retrieval Into Our Chain

Now that we have our loaded retriever component as a chain, we can incorporate it into our existing chat system as before. Specifically, we can start with an ***always-on RAG formulation*** where:
- **A retriever is always retrieving context by default**.
- **A generator is acting on the retrieved context**.

In [8]:
from langchain.document_transformers import LongContextReorder
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain.schema.runnable.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter

########################################################################
## Utility Runnables/Methods
def RPrint(preface=""):
    """Simple passthrough "prints, then returns" chain"""
    def print_and_return(x, preface):
        if preface: print(preface, end="")
        pprint(x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name:
            out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

## Optional; Reorders longer documents to center of output text
long_reorder = RunnableLambda(LongContextReorder().transform_documents)

In [9]:
context_prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context"
    "\n\nRetrieved Context: {context}"
    "\n\nUser Question: {question}"
    "\nAnswer the user conversationally. User is not aware of context."
)

chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'question': (lambda x:x)
    }
    | context_prompt
    | RPrint()
    | instruct_llm
    | StrOutputParser()
)

pprint(chain.invoke("Where does Beras live?"))

ChatPromptValue(
    messages=[
        HumanMessage(
            content="Answer the question using only the context\n\nRetrieved Context: [Quote from Document] [Beras]
I live in the arctic, so I'm not used to the warm climate there. I was just curious, ya know!\n[Quote from 
Document] [Beras] Wow, that sounds amazing! Ive never been to the Rocky Mountains before, but Ive heard many great 
things about them.\n[Quote from Document] [Agent] I hope you get to visit them someday, Beras! It would be a great 
adventure for you!\n[Quote from Document] [User]  Hello! My name is Beras, and I'm a big blue bear! Can you please 
tell me about the rocky mountains?\n\n\nUser Question: Where does Beras live?\nAnswer the user conversationally. 
User is not aware of context.",
            additional_kwargs={},
            response_metadata={}
        )
    ]
)

Okay, so I need to figure out where Beras lives. The user is asking a question and wants a conversational answer 
based on the context provided. Let me look through the context again.

First quote: Beras mentions, "I live in the arctic, so I'm not used to the warm climate there." Hmm, that's a 
direct statement about his residence. So, he lives in the Arctic.

Next, there's another quote where Beras expresses curiosity about the Rocky Mountains, saying he's never been there
but heard good things. The agent encourages him to visit, and the user introduces Beras as a big blue bear.

So, putting it all together, the first quote clearly states that Beras lives in the Arctic. The other information 
is about his interests or plans but not about where he currently lives. Therefore, the answer should be 
straightforward.
</think>

Beras lives in the Arctic.

Take a second to try out some more invocations and see how the new setup performs. Regardless of your model choice, the following questions should serve as interesting starting points.

In [10]:
pprint(chain.invoke("Where are the Rocky Mountains?"))

ChatPromptValue(
    messages=[
        HumanMessage(
            content="Answer the question using only the context\n\nRetrieved Context: [Quote from Document] [User] 
Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the rocky mountains?\n[Quote from 
Document] [Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains and their 
significance!\n[Quote from Document] [Agent] In the meantime, you can learn more about the Rocky Mountains by doing
some research online or watching documentaries about them.\n[Quote from Document] [Agent] The Rocky Mountains are a
beautiful and majestic range of mountains that stretch across North America\n\n\nUser Question: Where are the Rocky
Mountains?\nAnswer the user conversationally. User is not aware of context.",
            additional_kwargs={},
            response_metadata={}
        )
    ]
)

Okay, so the user is asking where the Rocky Mountains are located. Looking at the context provided, the Agent 
mentioned that the Rocky Mountains are a beautiful and majestic range stretching across North America. There's no 
specific details about exactly which countries or states, but I can use general knowledge to fill in the gaps. The 
Rocky Mountains stretch from Canada down through the United States into Mexico. In Canada, they cover provinces 
like British Columbia and Alberta. In the U.S., they pass through states including Idaho, Montana, Wyoming, 
Colorado, and New Mexico. The context doesn't specify, but adding that info would make the answer more helpful. I 
should keep it conversational and friendly, mentioning that they stretch across North America and provide some 
notable areas without making it too lengthy.
</think>

The Rocky Mountains are situated in North America, stretching from Canada through the United States and into 
Mexico. They pass through regions like British Columbia and Alberta in Canada, and states such as Idaho, Montana, 
Wyoming, Colorado, and New Mexico in the U.S.

In [11]:
pprint(chain.invoke("Where are the Rocky Mountains? Are they close to California?"))

ChatPromptValue(
    messages=[
        HumanMessage(
            content="Answer the question using only the context\n\nRetrieved Context: [Quote from Document] [Agent]
In the meantime, you can learn more about the Rocky Mountains by doing some research online or watching 
documentaries about them.\n[Quote from Document] [Agent] Absolutely! Lets continue the conversation and explore 
more about the Rocky Mountains and their significance!\n[Quote from Document] [User]  Hello! My name is Beras, and 
I'm a big blue bear! Can you please tell me about the rocky mountains?\n[Quote from Document] [Agent] The Rocky 
Mountains are a beautiful and majestic range of mountains that stretch across North America\n\n\nUser Question: 
Where are the Rocky Mountains? Are they close to California?\nAnswer the user conversationally. User is not aware 
of context.",
            additional_kwargs={},
            response_metadata={}
        )
    ]
)

Okay, so the user is asking where the Rocky Mountains are and if they're close to California. I remember from the 
context that the agent mentioned the Rockies stretch across North America. But I need to recall exactly where they 
go. I think they start in Canada and go through the United States as well. Wait, let me get this straight. They 
probably run through a few states. Colorado is one, Montana, Wyoming, maybe Idaho? I'm not entirely sure about all 
the states they pass through, but I know they're in the Western part of the US.

Now, about their proximity to California. California is on the West Coast, right? The Rockies are also in the west 
but more towards the interior. I think between the Rockies and California, there's the Great Basin, which includes 
mountain ranges like the Sierra Nevada. So the Rockies are not directly next to California. There's some distance 
between them, maybe a few hundred miles or more, depending on where exactly you're measuring.

I should explain that while they're both in the west, they're not right next to each other. Also, maybe mention 
that the Rockies are part of the larger North American Cordillera system, which includes other ranges too. The user
is a big blue bear named Beras, sokeeping the tone friendly and informative would be good.

Wait, in the context provided, the agent only gave a brief description, so I shouldn't make up more details than 
what's in the context, but maybe assume based on common knowledge since the user might not know where they are 
exactly. I should probably avoid technical terms and keep it simple. So summing up, the Rockies are in western 
North America, stretching from Canada through several US states, and while they are in the same general region as 
California, they're not close to it. There's a significant distance separating them, with other geographic features
in between.
</think>

The Rocky Mountains are located inwestern North America, stretching from Canada through several U.S. states like 
Montana, Wyoming, and Colorado. While they are in the same general region as California, they are not close to it. 
There is a considerable distance between the Rockies and California, with other geographic areas in between, such 
as the Great Basin and the Sierra Nevada range.

In [12]:
pprint(chain.invoke("How far away is Beras from the Rocky Mountains?"))

ChatPromptValue(
    messages=[
        HumanMessage(
            content="Answer the question using only the context\n\nRetrieved Context: [Quote from Document] [Beras]
Wow, that sounds amazing! Ive never been to the Rocky Mountains before, but Ive heard many great things about 
them.\n[Quote from Document] [Beras] I live in the arctic, so I'm not used to the warm climate there. I was just 
curious, ya know!\n[Quote from Document] [Agent] The Rocky Mountains are a beautiful and majestic range of 
mountains that stretch across North America\n[Quote from Document] [User]  Hello! My name is Beras, and I'm a big 
blue bear! Can you please tell me about the rocky mountains?\n\n\nUser Question: How far away is Beras from the 
Rocky Mountains?\nAnswer the user conversationally. User is not aware of context.",
            additional_kwargs={},
            response_metadata={}
        )
    ]
)

Alright, let's try to figure out how far Beras is from the Rocky Mountains. First, I see that the context mentions 
Beras living in the Arctic. The user provided some quotes where Beras talks about never visiting the Rockies but 
living in the Arctic. So, Beras is Arctic, Rockies are in North America.

I know the Arctic region includes areas like northern Canada, Alaska, and parts of Russia and Scandinavia. The 
Rocky Mountains stretch from northern British Columbia through the western U.S. So, if Beras is in, say, northern 
Canada or Alaska, the distance would be quite large.

I remember that the distance from the Arctic Circle to the Rockies can vary. For example, the distance from Barrow,
Alaska, to Denver, Colorado, is about 1,500 miles. But depending on where exactly Beras is, it could be more or 
less.

However, since the context doesn't specify the exact location within the Arctic, it's hard to give an exact figure.
It's safe to say that it's a significant distance, likely in the thousands of miles range if moving over land.

So, in simple terms, Beras is pretty far away from the Rockies. But without a specific starting point, I can't 
pinpoint the exact distance in miles. It's probably several thousand miles if traveling by land, considering the 
Rockies are in the central-western part of North America and the Arctic is much further north.
</think>

Beras lives in the Arctic, which is quite far from the Rocky Mountains in North America. The exact distance isn't 
clear, but it's likely several thousand miles away if traveling over land.

<br>

You might notice some decent performance with this always-on retrieval node in the loop since the actual context being fed into the LLM remains relatively small. It's important to experiment with factors like embedding sizes, context limits, and model options to see what kinds of behavior you can expect and which efforts are worth taking to improve performance.

<br>

### **Step 4:** Automatic Conversation Storage

Now that we see how our vector store memory unit should function, we can perform one last integration to allow our conversation to add new entries to our conversation: a runnable that calls the `add_texts` method for us to update the store state.


In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

########################################################################
## Reset knowledge base and define what it means to add more messages.
convstore = FAISS.from_texts(conversation, embedding=embedder)

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([f"User said {d.get('input')}", f"Agent said {d.get('output')}"])
    return d.get('output')

########################################################################

# instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")

chat_prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context"
    "\n\nRetrieved Context: {context}"
    "\n\nUser Question: {input}"
    "\nAnswer the user conversationally. Make sure the conversation flows naturally.\n"
    "[Agent]"
)


conv_chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'input': (lambda x:x)
    }
    | RunnableAssign({'output' : chat_prompt | instruct_llm | StrOutputParser()})
    | partial(save_memory_and_get_output, vstore=convstore)
)

pprint(conv_chain.invoke("I'm glad you agree! I can't wait to get some sweets there! It's such a good food!"))
print()
pprint(conv_chain.invoke("Can you guess what my favorite food is?"))
print()
pprint(conv_chain.invoke("Actually, my favorite is honey! Not sure where you got that idea?"))
print()
pprint(conv_chain.invoke("I see! Fair enough! Do you know my favorite food now?"))

Alright, so I need to figure out how to answer the user's message based on the provided context. The user 
mentioned, "I'm glad you agree! I can't wait to get some sweets there! It's such a good food!" 

Looking at the context, Beras talked about the Rocky Mountains and mentioned the warm climate, but there's nothing 
directly about sweets or food there. The agent previously responded positively about the Rockies and expressed hope
for Beras to visit.

Since there's no info on sweets in the context, the response should focus on the trip to the Rockies, maybe the 
experiences there, but without bringing up sweets. So the agent should acknowledge the excitement, perhaps talk 
about the adventure, and encourage Beras to plan the trip.

The response should be friendly and conversational. So maybe something like, "That sounds like a wonderful way to 
enjoy the trip! The Rockies are full of amazing experiences. I hope you get to go soon and make unforgettable 
memories there!"
</think>

That sounds like a wonderful way to enjoy the trip! The Rockies are full of amazing experiences. I hope you get to 
go soon and make unforgettable memories there!

Alright, let's see. The user just asked me to guess their favorite food. Hmm, the context they provided isn't 
giving me a direct answer, but I can take a few clues from it.

First, I recall that in the previous conversation, the user mentioned something about sweets when we were talking 
about the Rocky Mountains. They said, "I can't wait to get some sweets there! It's such a good food!" That seems 
like a hint that they might have a preference for sweets.

Also, the user is Beras, a big blue bear from the Arctic. Now, polar bears typically don't have access to a lot of 
sweets in their natural habitat, but they do enjoy eating seal blubber, which is high in fat and provides a lot of 
energy. But that's a guess based on what I know about polar bears in general, not specifically about Beras.

I want to be friendly and conversational, so maybe I should acknowledge the mention of sweets and also play on the 
idea of their diet as a bear. But without more context, it's a bit tricky. 

Alternatively, maybe Beras has a sweet tooth since they mentioned sweets in the Rockies context. It could be 
something they enjoy outside their usual diet. So, I might suggest sweets as a favorite, but also leave room for 
them to correct me if I'm wrong.

Putting it all together, I should make a light-hearted guess based on the sweets mention and perhaps hint at their 
nature as a bear, but without being too pushy. That way, the conversation remains engaging and open for the user to
elaborate if they choose to.
</think>

Well, based on your earlier mention of sweets in the Rockies conversation, I might guess that you have a sweet 
tooth! But since you're a big blue bear from the Arctic, maybe you have a taste for seal blubber? Just 
kidding—probably not! Or maybe something like berries or fish?iddlediil You'll have to tell me!What's your favorite

Alright, I need to respond to the user who corrected my earlier guess about their favorite food. In the history, 
they mentioned sweets and when I made a guess, they told me it's honey. 

First, I want to acknowledge their correction and show that I'm happy they clarified. Even though I mentioned seal 
blubber humorously, they clarified it's honey. I should express that I'm glad they told me and perhaps link it to 
their Arctic background. Maybe mention bees not living there but bears loving honey as a joke. End by asking for 
more details or something else to keep the conversation going.

I should make sure the tone is friendly and conversational, not too formal. Maybe use an emoji to keep it light, 
like a bear or honey face. Also, make the response flow smoothly so it feels natural.
</think>

Ah, got it! I knew you'd correct me—ha! That makes sense though, bears do love honey, even if there are no bees in 
the Arctic. 🐻🍯 So, do you prefer honey in a specific form, or just straight up from the hive? 😄

Alright, let's break this down. The user mentioned that I guessed their favorite food based on our previous 
conversation where they talked about sweets and their Arctic background as a bear. I initially guessed seal blubber
humorously, and they corrected me by saying it's honey. 

Now, the user is asking, "I see! Fair enough! Do you know my favorite food now?" I need to respond in a friendly 
and conversational way. I should acknowledge that they've told me and express happiness about knowing their 
favorite. 

I want to keep the tone light, maybe add an emoji to keep it engaging. Also, I should encourage them to share more 
or ask a follow-up question to keep the conversation going. Perhaps something like asking if they have a particular
way they enjoy honey or if there are any fun stories about their love for it.

Let me structure this in a way that flows naturally and maintains a warm, chatty tone.
</think>

Well, it seems you've made it clear—your favorite food is honey! 🍯 And actually, that makes perfect sense, given 
how much bears (and let's be honest, humans) love a good drizzle of honey on just about anything. It’s a great 
choice, and I’m glad you’ve shared that with me. So, do you have a particular way you enjoy your honey, or is it 
just straight from the hive? 😄

Unlike the more automatic full-text or rule-based approaches to injecting context into the LLM, this approach ensures some amount of consolidation which can keep the context length from getting out of hand. It's still not a full-proof strategy on its own, but it's a stark improvement for unstructured conversations (and doesn't even require a strong instruction-tuned model to perform slot-filling).

----

<br>

## **Part 3 [Exercise]:** RAG For Document Chunk Retrieval

Given our prior exploration of document loading, the idea that data chunks can be embedded and searched through probably isn't surprising. With that said, it is definitely worth going over since applying RAG with documents is a double-edged sword; it may **seem** to work well out of the box but requires some extra care when optimizing it for truly reliable performance. It also provides an excellent opportunity to review some fundamental LCEL skills, so let's see what we can do!

<br>

### **Exercise:**

In the previous example, you may recall that we pulled in some relatively small papers with the help of [`ArxivLoader`](https://python.langchain.com/docs/integrations/document_loaders/arxiv) using the following syntax:

```python
from langchain.document_loaders import ArxivLoader

docs = [
    ArxivLoader(query="2205.00445").load(),  ## MRKL
    ArxivLoader(query="2210.03629").load(),  ## ReAct
]
```

Given all that you've learned so far, choose a selection of papers that you would like to use and develop a chatbot that can talk about them!

<br>

Though this is a pretty big task, a walkthrough of ***most*** of the process will be provided below. By the end of the walkthrough, many of the necessary puzzle pieces will be provided, and your real task will be to integrate them together for the final `retrieval_chain`. When you're done, get ready to re-integrate the chain (or a flavor of your choice) in the last notebook as part of the evaluation exercise!


<br>

### **Task 1**: Loading And Chunking Your Documents

The following code block gives you some default papers to load in for your RAG chain. Feel free to select more papers as desired, but note that longer documents will take longer to process. A few simplifying assumptions and additional processing steps are included to help you improve your naive RAG performance:

- Documents are cut off prior to the "References" section if one exists. This will keep our system from considering the citations and appendix sections, which tend to be long and distracting.

- A chunk that lists the available documents is inserted to provide a high-level view of all available documents in a single chunk. If your pipeline does not provide metadata on each retrieval, this is a useful component and can even be listed among a list of higher-priority pieces if appropriate.

- Additionally, the metadata entries are also inserted to provide general information. Ideally, there would also be some synthetic chunks that merge the metadata into interesting cross-document chunks.

**NOTE:** ***For the sake of the assessment, please include at least one paper that is less than one month old!***


In [14]:
import json
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import ArxivLoader

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " "],
)

## TODO: Please pick some papers and add them to the list as you'd like
## NOTE: To re-use for the final assessment, make sure at least one paper is < 1 month old
print("Loading Documents")
docs = [
    ArxivLoader(query="1706.03762").load(),  ## Attention Is All You Need Paper
    ArxivLoader(query="1810.04805").load(),  ## BERT Paper
    ArxivLoader(query="2005.11401").load(),  ## RAG Paper
    ArxivLoader(query="2205.00445").load(),  ## MRKL Paper
    ArxivLoader(query="2310.06825").load(),  ## Mistral Paper
    ArxivLoader(query="2306.05685").load(),  ## LLM-as-a-Judge
    ArxivLoader(query="2503.07275").load(),  #Automatic Curriculum Design for Zero-Shot Human-AI Coordination
    ## Some longer papers
    # ArxivLoader(query="2210.03629").load(),  ## ReAct Paper
    # ArxivLoader(query="2112.10752").load(),  ## Latent Stable Diffusion Paper
    # ArxivLoader(query="2103.00020").load(),  ## CLIP Paper
    ## TODO: Feel free to add more
]

## Cut the paper short if references is included.
## This is a standard string in papers.
for doc in docs:
    content = json.dumps(doc[0].page_content)
    if "References" in content:
        doc[0].page_content = content[:content.index("References")]

## Split the documents and also filter out stubs (overly short chunks)
print("Chunking Documents")
docs_chunks = [text_splitter.split_documents(doc) for doc in docs]
docs_chunks = [[c for c in dchunks if len(c.page_content) > 200] for dchunks in docs_chunks]

## Make some custom Chunks to give big-picture details
doc_string = "Available Documents:"
doc_metadata = []
for chunks in docs_chunks:
    metadata = getattr(chunks[0], 'metadata', {})
    doc_string += "\n - " + metadata.get('Title')
    doc_metadata += [str(metadata)]

extra_chunks = [doc_string] + doc_metadata

## Printing out some summary information for reference
pprint(doc_string, '\n')
for i, chunks in enumerate(docs_chunks):
    print(f"Document {i}")
    print(f" - # Chunks: {len(chunks)}")
    print(f" - Metadata: ")
    pprint(chunks[0].metadata)
    print()

Loading Documents
Chunking Documents


Available Documents:
 - Attention Is All You Need
 - BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding
 - Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
 - MRKL Systems: A modular, neuro-symbolic architecture that combines large language models, external knowledge 
sources and discrete reasoning
 - Mistral 7B
 - Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena
 - Automatic Curriculum Design for Zero-Shot Human-AI Coordination 

Document 0
 - # Chunks: 35
 - Metadata: 


{
    'Published': '2023-08-02',
    'Title': 'Attention Is All You Need',
    'Authors': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz 
Kaiser, Illia Polosukhin',
    'Summary': 'The dominant sequence transduction models are based on complex recurrent or\nconvolutional neural 
networks in an encoder-decoder configuration. The best\nperforming models also connect the encoder and decoder 
through an attention\nmechanism. We propose a new simple network architecture, the Transformer, based\nsolely on 
attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation
tasks show these models to be\nsuperior in quality while being more parallelizable and requiring 
significantly\nless time to train. Our model achieves 28.4 BLEU on the WMT 2014\nEnglish-to-German translation 
task, improving over the existing best results,\nincluding ensembles by over 2 BLEU. On the WMT 2014 
English-to-French\ntranslation task, our model establishes a new single-model state-of-the-art\nBLEU score of 41.8 
after training for 3.5 days on eight GPUs, a small fraction\nof the training costs of the best models from the 
literature. We show that the\nTransformer generalizes well to other tasks by applying it successfully to\nEnglish 
constituency parsing both with large and limited training data.'
}


Document 1
 - # Chunks: 45
 - Metadata: 


...

{
    'Published': '2023-10-10',
    'Title': 'Mistral 7B',
    'Authors': 'Albert Q. Jiang, Alexandre Sablayrolles, Arthur Mensch, Chris Bamford, Devendra Singh Chaplot, 
Diego de las Casas, Florian Bressand, Gianna Lengyel, Guillaume Lample, Lucile Saulnier, Lélio Renard Lavaud, 
Marie-Anne Lachaux, Pierre Stock, Teven Le Scao, Thibaut Lavril, Thomas Wang, Timothée Lacroix, William El Sayed',
    'Summary': 'We introduce Mistral 7B v0.1, a 7-billion-parameter language model engineered\nfor superior 
performance and efficiency. Mistral 7B outperforms Llama 2 13B\nacross all evaluated benchmarks, and Llama 1 34B in
reasoning, mathematics, and\ncode generation. Our model leverages grouped-query attention (GQA) for 
faster\ninference, coupled with sliding window attention (SWA) to effectively handle\nsequences of arbitrary length
with a reduced inference cost. We also provide a\nmodel fine-tuned to follow instructions, Mistral 7B -- Instruct, 
that surpasses\nthe Llama 2 13B -- Chat model both on human and automated benchmarks. Our\nmodels are released 
under the Apache 2.0 license.'
}


Document 5
 - # Chunks: 44
 - Metadata: 


{
    'Published': '2023-12-24',
    'Title': 'Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena',
    'Authors': 'Lianmin Zheng, Wei-Lin Chiang, Ying Sheng, Siyuan Zhuang, Zhanghao Wu, Yonghao Zhuang, Zi Lin, 
Zhuohan Li, Dacheng Li, Eric P. Xing, Hao Zhang, Joseph E. Gonzalez, Ion Stoica',
    'Summary': 'Evaluating large language model (LLM) based chat assistants is challenging\ndue to their broad 
capabilities and the inadequacy of existing benchmarks in\nmeasuring human preferences. To address this, we explore
using strong LLMs as\njudges to evaluate these models on more open-ended questions. We examine the\nusage and 
limitations of LLM-as-a-judge, including position, verbosity, and\nself-enhancement biases, as well as limited 
reasoning ability, and propose\nsolutions to mitigate some of them. We then verify the agreement between 
LLM\njudges and human preferences by introducing two benchmarks: MT-bench, a\nmulti-turn question set; and Chatbot 
Arena, a crowdsourced battle platform. Our\nresults reveal that strong LLM judges like GPT-4 can match both 
controlled and\ncrowdsourced human preferences well, achieving over 80% agreement, the same\nlevel of agreement 
between humans. Hence, LLM-as-a-judge is a scalable and\nexplainable way to approximate human preferences, which 
are otherwise very\nexpensive to obtain. Additionally, we show our benchmark and traditional\nbenchmarks complement
each other by evaluating several variants of LLaMA and\nVicuna. The MT-bench questions, 3K expert votes, and 30K 
conversations with\nhuman preferences are publicly available 
at\nhttps://github.com/lm-sys/FastChat/tree/main/fastchat/llm_judge.'
}


Document 6
 - # Chunks: 36
 - Metadata: 


{
    'Published': '2025-03-10',
    'Title': 'Automatic Curriculum Design for Zero-Shot Human-AI Coordination',
    'Authors': 'Won-Sang You, Tae-Gwan Ha, Seo-Young Lee, Kyung-Joong Kim',
    'Summary': "Zero-shot human-AI coordination is the training of an ego-agent to coordinate\nwith humans without 
using human data. Most studies on zero-shot human-AI\ncoordination have focused on enhancing the ego-agent's 
coordination ability in\na given environment without considering the issue of generalization to 
unseen\nenvironments. Real-world applications of zero-shot human-AI coordination should\nconsider unpredictable 
environmental changes and the varying coordination\nability of co-players depending on the environment. Previously,
the multi-agent\nUED (Unsupervised Environment Design) approach has investigated these\nchallenges by jointly 
considering environmental changes and co-player policy in\ncompetitive two-player AI-AI scenarios. In this paper, 
our study extends the\nmulti-agent UED approach to a zero-shot human-AI coordination. We propose a\nutility 
function and co-player sampling for a zero-shot human-AI coordination\nsetting that helps train the ego-agent to 
coordinate with humans more\neffectively than the previous multi-agent UED approach. The zero-shot 
human-AI\ncoordination performance was evaluated in the Overcooked-AI environment, using\nhuman proxy agents and 
real humans. Our method outperforms other baseline\nmodels and achieves a high human-AI coordination performance in
unseen\nenvironments."
}

<br>

### **Task 2**: Construct Your Document Vector Stores

Now that we have all of the components, we can go ahead and create indices surrounding them:

In [15]:
%%time
print("Constructing Vector Stores")
vecstores = [FAISS.from_texts(extra_chunks, embedder)]
vecstores += [FAISS.from_documents(doc_chunks, embedder) for doc_chunks in docs_chunks]

Constructing Vector Stores
CPU times: user 791 ms, sys: 61.6 ms, total: 852 ms
Wall time: 21.2 s


<br>

From there, we can combine our indices into a single one using the following utility:

In [16]:
from faiss import IndexFlatL2
from langchain_community.docstore.in_memory import InMemoryDocstore

embed_dims = len(embedder.embed_query("test"))
def default_FAISS():
    '''Useful utility for making an empty FAISS vectorstore'''
    return FAISS(
        embedding_function=embedder,
        index=IndexFlatL2(embed_dims),
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
        normalize_L2=False
    )

def aggregate_vstores(vectorstores):
    ## Initialize an empty FAISS Index and merge others into it
    ## We'll use default_faiss for simplicity, though it's tied to your embedder by reference
    agg_vstore = default_FAISS()
    for vstore in vectorstores:
        agg_vstore.merge_from(vstore)
    return agg_vstore

## Unintuitive optimization; merge_from seems to optimize constituent vector stores away
docstore = aggregate_vstores(vecstores)

print(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")

Constructed aggregate docstore with 275 chunks


<br>

### **Task 3: [Exercise]** Implement Your RAG Chain

Finally, all the puzzle pieces are in place to implement the RAG pipeline! As a review, we now have:

- A way to construct a from-scratch vector store for conversational memory (and a way to initialize an empty one with `default_FAISS()`)

- A vector store pre-loaded with useful document information from our `ArxivLoader` utility (stored in `docstore`).

With the help of a couple more utilities, you're finally ready to integrate your chain! A few additional convenience utilities are provided (`doc2str` and the now-common `RPrint`) but are optional to use. Additionally, some starter prompts and structures are also defined.

> **Given all of this:** Please implement the `retrieval_chain`.

In [17]:
from langchain.document_transformers import LongContextReorder
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

import gradio as gr
from functools import partial
from operator import itemgetter

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")
# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")
# instruct_llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")

convstore = default_FAISS()

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([
        f"User previously responded with {d.get('input')}",
        f"Agent previously responded with {d.get('output')}"
    ])
    return d.get('output')

initial_msg = (
    "Hello! I am a document chat agent here to help the user!"
    f" I have access to the following documents: {doc_string}\n\nHow can I help you?"
)

chat_prompt = ChatPromptTemplate.from_messages([("system",
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked: {input}\n\n"
    " From this, we have retrieved the following potentially-useful info: "
    " Conversation History Retrieval:\n{history}\n\n"
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational.)"
), ('user', '{input}')])

stream_chain = chat_prompt| RPrint() | instruct_llm | StrOutputParser()

################################################################################################
## BEGIN TODO: Implement the retrieval chain to make your system work!

retrieval_chain = (
    {'input' : (lambda x: x)}
    ## TODO: Make sure to retrieve history & context from convstore & docstore, respectively.
    ## HINT: Our solution uses RunnableAssign, itemgetter, long_reorder, and docs2str
    | RunnableAssign({'history' : lambda d: itemgetter('input') | convstore.as_retriever() | long_reorder | docs2str})
    | RunnableAssign({'context' : lambda d: itemgetter('input') | docstore.as_retriever() | long_reorder | docs2str})
)

## END TODO
################################################################################################

def chat_gen(message, history=[], return_buffer=True):
    buffer = ""
    ## First perform the retrieval based on the input message
    retrieval = retrieval_chain.invoke(message)
    line_buffer = ""

    ## Then, stream the results of the stream_chain
    for token in stream_chain.stream(retrieval):
        buffer += token
        ## If you're using standard print, keep line from getting too long
        yield buffer if return_buffer else token

    ## Lastly, save the chat exchange to the conversation memory buffer
    save_memory_and_get_output({'input':  message, 'output': buffer}, convstore)


## Start of Agent Event Loop
test_question = "Tell me about paper made by Koreans"  ## <- modify as desired

## Before you launch your gradio interface, make sure your thing works
for response in chat_gen(test_question, return_buffer=False):
    print(response, end='')

ChatPromptValue(
    messages=[
        SystemMessage(
            content='You are a document chatbot. Help the user as they ask questions about documents. User messaged
just asked: Tell me about paper made by Koreans\n\n From this, we have retrieved the following potentially-useful 
info:  Conversation History Retrieval:\n\n\n Document Retrieval:\n[Quote from Automatic Curriculum Design for 
Zero-Shot Human-AI Coordination] "Automatic Curriculum Design for Zero-Shot Human-AI Coordination\\nWon-Sang You1 ,
Tae-Gwan Ha2 , Seo-Young Lee1 and Kyung-Joong Kim1\\n1Department of AI Convergence, Gwangju Institute of Science 
and Technology,\\n2School of Integrated Technology, Gwangju Institute of Science and Technology\\n{u.wonsang0514, 
hataegwan, seoyoung.john}@gm.gist.ac.kr, kjkim@gist.ac.kr\\nAbstract\\nZero-shot human-AI coordination is the 
training\\nof an ego-agent to coordinate with humans with-\\nout using human data.\\nMost studies on zero-\\nshot 
human-AI coordination have focused on en-\\nhancing the ego-agent\\u2019s coordination ability in a\\ngiven 
environment without considering the issue\\nof generalization to unseen environments.\\nReal-\\nworld applications 
of zero-shot human-AI coordi-\\nnation should consider unpredictable environmen-\\ntal changes and the varying 
coordination ability of\\nco-players depending on the environment\n[Quote from BERT: Pre-training of Deep 
Bidirectional Transformers for Language Understanding] . The contributions\\nof our paper are as follows:\\n\\u2022
We demonstrate the importance of bidirectional\\npre-training for language representations. Un-\\nlike Radford et 
al. (2018), which uses unidirec-\\ntional language models for pre-training, BERT\\nuses masked language models to 
enable pre-\\ntrained deep bidirectional representations. This\\nis also in contrast to Peters et al. (2018a), 
which\\nuses a shallow concatenation of independently\\ntrained left-to-right and right-to-left LMs.\\n\\u2022 We 
show that pre-trained representations reduce\\nthe need for many heavily-engineered task-\\nspeci\\ufb01c 
architectures. BERT is the \\ufb01rst \\ufb01ne-\\ntuning based representation model that 
achieves\\nstate-of-the-art performance on a large suite\\nof sentence-level and token-level tasks, 
outper-\\nforming many task-speci\\ufb01c architectures.\\n\\u2022 BERT advances the state of the art for 
eleven\\nNLP tasks.\\nThe code and pre-trained mod-\\nels are available at https://github\n[Quote from Automatic 
Curriculum Design for Zero-Shot Human-AI Coordination] . Previ-\\nously, the multi-agent UED (Unsupervised 
Envi-\\nronment Design) approach has investigated these\\nchallenges by jointly considering environmental\\nchanges
and co-player policy in competitive two-\\nplayer AI-AI scenarios. In this paper, our study\\nextends the 
multi-agent UED approach to a zero-\\nshot human-AI coordination. We propose a util-\\nity function and co-player 
sampling for a zero-\\nshot human-AI coordination setting that helps train\\nthe ego-agent to coordinate with 
humans more ef-\\nfectively than the previous multi-agent UED ap-\\nproach. The zero-shot human-AI coordination 
per-\\nformance was evaluated in the Overcooked-AI en-\\nvironment, using human proxy agents and real 
hu-\\nmans.\\nOur method outperforms other baseline\\nmodels and achieves a high human-AI 
coordination\\nperformance in unseen environments.\\n1\\nIntroduction\\nDeep reinforcement learning has 
demonstrated success across\\ndiverse domains such as gaming [Silver et al., 2017; Berner\\net al\n[Quote from MRKL
Systems: A modular, neuro-symbolic architecture that combines large language models, external knowledge sources and
discrete reasoning] . We describe this\\nneuro-symbolic architecture, dubbed the Modular Reasoning, Knowledge and 
Lan-\\nguage (MRKL, pronounced \\u201cmiracle\\u201d) system, some of the technical challenges in\\nimplementing 
it, and Jurassic-X, AI21 Labs\\u2019 MRKL system implementation.\

Sure, I can provide some information about a paper that was authored by Koreans. The paper I found is called "Automatic Curriculum Design for Zero-Shot Human-AI Coordination." It was written by Won-Sang You, Tae-Gwan Ha, Seo-Young Lee, and Kyung-Joong Kim from the Gwangju Institute of Science and Technology in Korea.

The paper focuses on the challenge of training an artificial intelligence (AI) system to coordinate with humans without using human data. The researchers propose a method for training the AI system using a curriculum design approach, which gradually increases the complexity of the tasks the AI system must perform. This approach allows the AI system to learn more effectively and to coordinate with humans more successfully.

The researchers evaluate their method in a cooking game environment, where the AI system must coordinate with human players to prepare meals. They find that their method outperforms other approaches and enables the AI system to coordinate effectively wi

### **Task 4:** Interact With Your Gradio Chatbot

In [18]:
# chatbot = gr.Chatbot(value = [[None, initial_msg]])
# demo = gr.ChatInterface(chat_gen, chatbot=chatbot).queue()

# try:
#     demo.launch(debug=True, share=True, show_api=False)
#     demo.close()
# except Exception as e:
#     demo.close()
#     print(e)
#     raise e

<br>

----

<br>

## **Part 4:** Saving Your Index For Evaluation

After you've implemented your RAG chain, please save your accumulated vector store as shown [in the official documentation](https://python.langchain.com/docs/integrations/vectorstores/faiss#saving-and-loading). You'll have a chance to use it again for your final assessment!

In [19]:
## Save and compress your index
docstore.save_local("docstore_index")
!tar czvf docstore_index.tgz docstore_index

!rm -rf docstore_index

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss


If everything was properly saved, the following line can be invoked to pull the index from the compressed `tgz` file (assuming the pip requirements are installed). After you have confirmed that the cell can pull in your index, download `docstore_index.tgz` for use in the last notebook!

In [20]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

# embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")
!tar xzvf docstore_index.tgz
new_db = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
docs = new_db.similarity_search("Testing the index")
print(docs[0].page_content[:1000])

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss
. To demonstrate, we build an index using the DrQA [5]\nWikipedia dump from December 2016 and compare outputs from RAG using this index to the newer\nindex from our main results (December 2018). We prepare a list of 82 world leaders who had changed\n7\nTable 4: Human assessments for the Jeopardy\nQuestion Generation Task.\nFactuality\nSpeci\ufb01city\nBART better\n7.1%\n16.8%\nRAG better\n42.7%\n37.4%\nBoth good\n11.7%\n11.8%\nBoth poor\n17.7%\n6.9%\nNo majority\n20.8%\n20.1%\nTable 5: Ratio of distinct to total tri-grams for\ngeneration tasks.\nMSMARCO\nJeopardy QGen\nGold\n89.6%\n90.0%\nBART\n70.7%\n32.4%\nRAG-Token\n77.8%\n46.8%\nRAG-Seq.\n83.5%\n53.8%\nTable 6: Ablations on the dev set. As FEVER is a classi\ufb01cation task, both RAG models are equivalent.\nModel\nNQ\nTQA\nWQ\nCT\nJeopardy-QGen\nMSMarco\nFVR-3\nFVR-2\nExact Match\nB-1\nQB-1\nR-L\nB-1\nLabel Accuracy\nRAG-Token-BM25\n29.7\n41.5\n32.1\n33.1\n17.5\n22

-----

<br>

## **Part 5:** Wrap-Up

Congratulations! Assuming your RAG chain is all good, you're now ready to move on to the **RAG Evaluation [Assessment]** section!

### <font color="#76b900">**Great Job!**</font>

### **Next Steps:**
1. **[Optional]** Revisit the **"Questions To Think About" Section** at the top of the notebook and think about some possible answers.

---

<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>